In [24]:
import sys
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
from matplotlib.ticker import PercentFormatter
import numpy as np
import math

In [45]:
#defining functions required:
def calculate_exponent(t_start, t_end, posn, l, r, u):                                   
    U = u*l
    a = g + r*posn                                                              
    b = g + r*(posn + l)                                                        
    E1 = (U/(r*l*(1-a))) * (1 + (a/((1-a)*(t_end-t_start))) * math.log((a+(t_start*(1-a)))/(a + (t_end*(1-a)))))
    E2 = -1.0*(U/(r*l*(1-b)))*(1 + (b/((1-b)*(t_end-t_start)))*math.log((b + ((1-b)*t_start))/(b + ((1-b)*t_end))))
    E = E1 + E2                                                                 
    return (E)                                                                  

In [46]:
def calculate_pi(posn, l, r, u):                                                         
    E_f1 = calculate_exponent(t1half, t2, posn, l, r, u)                                 
    E_f2 = calculate_exponent(t2, t3, posn, l, r, u)                                     
    E_f3 = calculate_exponent(t3, t4, posn, l, r, u)                                     
    pi_posn = f0*pi + f1*0.5*pi + f1*0.5*pi*math.exp(-1.0*E_f1) + f2*pi*math.exp(-1.0*E_f2) + f3*pi*math.exp(-1.0*E_f3)
    return (pi_posn)

In [47]:
def fit_log_curve(l_posns, l_pi):                                               
    x_data = np.array(l_posns)                                                  
    y_data = np.array(l_pi)                                                     
    log_x_data = np.log(x_data)                                                 
    curve_fit = np.polyfit(log_x_data, y_data, 1)                               
    return (curve_fit[0], curve_fit[1])

In [48]:
def calculate_pi_segment(posn_start, posn_end, l, r, u):
    l_posns, l_pi = [], []                                                          
    j=int(posn_start)                                                               
    while j <= int(posn_end):                                                           
        l_posns.append(j)                                                           
        l_pi.append(calculate_pi(j, l, r, u))                                                
        j += 1                                                                      
    return(l_posns, l_pi)

In [90]:
#Load in recombination data
df = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/GRCh37_exons_recRates.txt", sep='\t', 
                 names=['chrom','start','end','length','pi50', 'strand', 'r', 'overlap'])
#Create df of duplicates (ie regions that cross recombination bins)
df2 = df[df.duplicated(['chrom','start','end'])].drop_duplicates(['chrom','start','end'])
#Create lists of various columns
chroms = list(df2.chrom)
starts = list(df2.start)
ends = list(df2.end)
lengths = list(df2.length)
pi = list(df2['pi50'])
strands = list(df2.strand)

In [91]:
res = []
#Loop through duplicates
for i,j in enumerate(chroms):
    #Create df of current duplicate
    tdf = df[(df.chrom==chroms[i]) & (df.start==starts[i]) & (df.end==ends[i])].copy()
    #Adjust recombination rate relative to the size of the coverage
    tdf['r'] = tdf['r'] * (tdf['overlap'] / tdf['length'])
    #Create list that will act as row in df and append
    res.append([chroms[i],starts[i],ends[i],lengths[i],pi[i],strands[i], tdf.r.mean(), tdf.overlap.sum()])
#Remove duplicates from main df    
df.drop_duplicates(['chrom','start','end'], inplace=True, keep=False)
#Create df from duplicated data
ddf = pd.DataFrame(res, columns=['chrom','start','end','length','pi50', 'strand', 'r', 'overlap'])
#Concatenate dfs
rdf = pd.concat([df, ddf])

#Create empty df to store sorted data
df = pd.DataFrame()
for i in range(1, 23):
    #Subset by chromosome, sort by start coordinate, then concatenate with df
    tdf = rdf[rdf.chrom == "chr" + str(i)]
    tdf = tdf.sort_values("start")
    df = pd.concat([df, tdf])
    
#Remove regions that don't have complete coverage
df = df[df.overlap >= df.length]

In [87]:
#Load in mutation data
mdf = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/GRCh37_exons_mutRates.txt", sep='\t', 
                 names=['chrom','start','end','length','pi50', 'strand', 'm', 'overlap'])
#Create df of duplicates (ie regions that cross recombination bins)
mdf2 = mdf[mdf.duplicated(['chrom','start','end'])].drop_duplicates(['chrom','start','end'])
#Create lists of various columns
chroms = list(mdf2.chrom)
starts = list(mdf2.start)
ends = list(mdf2.end)
lengths = list(mdf2.length)
pi = list(df2['pi50'])
strands = list(mdf2.strand)

In [88]:
res = []
#Loop through duplicates
for i,j in enumerate(chroms):
    #Create df of current duplicate
    tdf = mdf[(mdf.chrom==chroms[i]) & (mdf.start==starts[i]) & (mdf.end==ends[i])].copy()
    #Adjust recombination rate relative to the size of the coverage
    tdf['m'] = tdf['m'] * (tdf['overlap'] / tdf['length'])
    #Create list that will act as row in df and append
    res.append([chroms[i],starts[i],ends[i],lengths[i],pi[i],strands[i], tdf.m.mean(), tdf.overlap.sum()])
#Remove duplicates from main df    
mdf.drop_duplicates(['chrom','start','end'], inplace=True, keep=False)
#Create df from duplicated data
ddf = pd.DataFrame(res, columns=['chrom','start','end','length','pi50', 'strand', 'r', 'overlap'])
#Concatenate dfs
rdf = pd.concat([mdf, ddf])

#Create empty df to store sorted data
mdf = pd.DataFrame()
for i in range(1, 23):
    #Subset by chromosome, sort by start coordinate, then concatenate with df
    tdf = rdf[rdf.chrom == "chr" + str(i)]
    tdf = tdf.sort_values("start")
    mdf = pd.concat([mdf, tdf])
    
#Remove regions that don't have complete coverage
mdf = mdf[mdf.overlap >= mdf.length]
mdf = mdf[['chrom','start','end','m']]
df = pd.merge(df, mdf, on=['chrom','start','end'], how='inner')
df = df.drop(columns=['overlap'])

In [92]:
#df['region'] = df.index + 1
len(df)
#df.to_csv(r"/home/vivak/human_demog_DFE/DFE_inference/GRCh37_exons_r_m.bed", sep='\t', header=True, index=False)

420

In [68]:
Ne = 20000
df['pi'] = 4 * Ne * df['m']
df.head()

,chrom,start,end,length,pi50,strand,r,m,region,u,pi
0,chr1,9427507,9429591,2084,2511,+,5.866640,1.719042e-08,1,6.876169e-08,0.001375
1,chr1,9908334,9910834,2500,718,-,0.000003,1.719042e-08,2,6.876169e-08,0.001375
2,chr1,16111043,16113084,2041,771,+,0.458002,1.341703e-08,3,5.366810e-08,0.001073
3,chr1,18807424,18812493,5069,2065,+,1.326100,2.143546e-08,4,8.574184e-08,0.001715
4,chr1,20878932,20881513,2581,1421,+,3.101520,2.032655e-08,5,8.130618e-08,0.001626


In [73]:
#Define variables and constants:                                                
g = 0.0 #rate of gene conversion                                                
f0 = 0.22 #(*Proportion of effectively neutral mutations with 0 <= |2Nes| < 1 *) 
f1 = 0.27 #(*Proportion of weakly deleterious mutations with 1 <= |2Nes| < 10 *) 
f2 = 0.13 #(*Proportion of moderately deleterious mutations with 10 <= |2Nes| < 100 *)
f3 = 0.38 #(*Proportion of strongly deleterious mutations with |2Nes| >= 100 *)  
#(*Note that the number of classes can easily be increased to whatever is required to approximate the continuous DFE *)
h = 0.5 #(* dominance coefficient *)
t0 = 0.0                                                                        
t1 = h*(1/(2*Ne))                                                               
t1half = h*(5/(2*Ne)) #(* This is the cut-off value of 2Nes=5. This derivation assumes that all mutations with 2Nes<5 will not contribute to BGS *)
t2 = h*(10/(2*Ne))                                                              
t3 = h*(100/(2*Ne))                                                             
t4 = h*1.0

#defining functions required:
def calculate_exponent(t_start, t_end, posn, l, r, u):                                   
    U = u*l
    a = g + r*posn                                                              
    b = g + r*(posn + l)                                                        
    E1 = (U/(r*l*(1-a))) * (1 + (a/((1-a)*(t_end-t_start))) * math.log((a+(t_start*(1-a)))/(a + (t_end*(1-a)))))
    E2 = -1.0*(U/(r*l*(1-b)))*(1 + (b/((1-b)*(t_end-t_start)))*math.log((b + ((1-b)*t_start))/(b + ((1-b)*t_end))))
    E = E1 + E2                                                                 
    return (E)                                                                  
                                                                                
def calculate_pi(posn, l, r, u, pi):                                                         
    E_f1 = calculate_exponent(t1half, t2, posn, l, r, u)                                 
    E_f2 = calculate_exponent(t2, t3, posn, l, r, u)                                     
    E_f3 = calculate_exponent(t3, t4, posn, l, r, u)                                     
    pi_posn = f0*pi + f1*0.5*pi + f1*0.5*pi*math.exp(-1.0*E_f1) + f2*pi*math.exp(-1.0*E_f2) + f3*pi*math.exp(-1.0*E_f3)
    return (pi_posn)

def fit_log_curve(l_posns, l_pi):                                               
    x_data = np.array(l_posns)                                                  
    y_data = np.array(l_pi)                                                     
    log_x_data = np.log(x_data)                                                 
    curve_fit = np.polyfit(log_x_data, y_data, 1)                               
    return (curve_fit[0], curve_fit[1])

def calculate_pi_segment(posn_start, posn_end, l, r, u, pi):
    l_posns, l_pi = [], []                                                          
    j=int(posn_start)                                                               
    while j <= int(posn_end):                                                           
        l_posns.append(j)                                                           
        l_pi.append(calculate_pi(j, l, r, u, pi))                                                
        j += 1                                                                      
    return(l_posns, l_pi)


In [79]:
numbp50 = []
numbp75 = []
for i in range(0, len(df)):
    pi_segment = calculate_pi_segment(1, 100000, df['length'].iloc[i], df.r.iloc[i]*1e-8, df.m.iloc[i], df.pi.iloc[i])
    log_fit_parameters = fit_log_curve(pi_segment[0], pi_segment[1])
    slope = log_fit_parameters[0]
    intercept = log_fit_parameters[1]
    try:
        pi_50 = float(calculate_pi(1, df['length'].iloc[i], df.r.iloc[i]*1e-8, df.m.iloc[i], df.pi.iloc[i])) + ((df.pi.iloc[i] - float(calculate_pi(1, df['length'].iloc[i], df.r.iloc[i]*1e-8, df.m.iloc[i], df.pi.iloc[i])))*0.5)
        numbp50.append(math.exp((pi_50-intercept)/slope))
    except OverflowError:
        numbp50.append("NA")
    try:
        pi_75 = float(calculate_pi(1, df['length'].iloc[i], df.r.iloc[i]*1e-8, df.m.iloc[i], df.pi.iloc[i])) + ((df.pi.iloc[i] - float(calculate_pi(1, df['length'].iloc[i], df.r.iloc[i]*1e-8, df.m.iloc[i], df.pi.iloc[i])))*0.75)
        numbp75.append(math.exp((pi_75-intercept)/slope))
    except OverflowError:
        numbp75.append("NA")    

/tmp/ipykernel_202367/3430400200.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  E1 = (U/(r*l*(1-a))) * (1 + (a/((1-a)*(t_end-t_start))) * math.log((a+(t_start*(1-a)))/(a + (t_end*(1-a)))))
/tmp/ipykernel_202367/3430400200.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  E2 = -1.0*(U/(r*l*(1-b)))*(1 + (b/((1-b)*(t_end-t_start)))*math.log((b + ((1-b)*t_start))/(b + ((1-b)*t_end))))
/tmp/ipykernel_202367/3430400200.py:23: RuntimeWarning: invalid value encountered in scalar add
  E = E1 + E2


In [84]:
len(df)

397

In [82]:
df.tail(10)

,chrom,start,end,length,pi50,strand,r,m,region,u,pi
387,chr20,23026270,23030301,4031,1603,-,8.555840,2.056852e-08,388,8.227407e-08,0.001645
388,chr20,34442450,34444728,2278,891,+,0.000086,1.528774e-08,389,6.115097e-08,0.001223
389,chr20,35219313,35222355,3042,2200,+,0.047282,1.528774e-08,390,6.115097e-08,0.001223
390,chr20,36718128,36720766,2638,3220,+,0.828422,1.823578e-08,391,7.294312e-08,0.001459
391,chr20,39314488,39317880,3392,6438,-,3.015720,2.117738e-08,392,8.470950e-08,0.001694
392,chr20,39830697,39833706,3009,1459,-,0.000000,1.840965e-08,393,7.363859e-08,0.001473
393,chr20,50003494,50007988,4494,661,-,7.681840,1.845788e-08,394,7.383153e-08,0.001477
394,chr21,28144265,28147084,2819,2032,-,1.838740,2.010357e-08,395,8.041430e-08,0.001608
395,chr22,28192751,28197486,4735,3506,-,16.025400,1.471311e-08,396,5.885242e-08,0.001177
396,chr22,42605657,42611347,5690,682,-,0.000000,1.406944e-08,397,5.627778e-08,0.001126
